In [3]:
# packages
using Distributions
using Random
using Plots
using Distances

In [4]:
# ackley function

function ackley(x)
    # x = [x1, x2, ..., xd]
    # a = default value 20
    # b = default value 0.2
    # c = default value 2*pi
    a = 20
    b = 0.2
    c = 2*π
    n = length(x)
    sum1 = 0
    sum2 = 0
    for i in 1:n
        sum1 += x[i]^2
        sum2 += cos(c*x[i])
    end
    
    term1 = -a * exp(-b*sqrt(sum1/n))
    term2 = -exp(sum2/n)
    y = term1 + term2 + a + exp(1)
    return y
end

ackley (generic function with 1 method)

In [5]:
# rastrigin function

function rastrigin(x)
    val = -10length(x)
    n = length(x)
    for i in 1:n
        val += -x[i]^2+10cos(π*x[i])
    end
    val =-val
    return val
end

rastrigin (generic function with 1 method)

In [6]:
# zirilli function

function zirilli2(x)
    x1 = x[1]
    x2 = x[2]
    val = (0.5 * (x1 ^ 2)) + (0.5 * (1 - cos(2*x1)) ) + (x2 ^ 2)
    return val
end

zirilli2 (generic function with 1 method)

In [7]:
# matyas function

function matyas2(x)
    x1 = x[1]
    x2 = x[2]
    val = (0.26 * ((x1 ^ 2) + (x2 ^ 2) ))  - (0.48 * x1 * x2)
    return val
end

matyas2 (generic function with 1 method)

In [8]:
# De Jong (sphere) function

sphere(x)=sum(x.^2)

sphere (generic function with 1 method)

In [11]:
# Schwefel 2.22 function

function schwefel_2_22(x)
    sum = 0
    prod = 1
    n = length(x)
    for i in 1:n
        sum += abs(x[i])
        prod *= abs(x[i])
    end
    val = sum+ prod
    return val
end

schwefel_2_22 (generic function with 1 method)

In [12]:
# population generator
function rand_population_uniform(n, d, a, b)
    # n = population size
    # a = lower bound
    # b = upper bound
    # d = dimensions
    return [convert(Array{Float64},rand(a:b,d)) for i in 1:n]
end

rand_population_uniform (generic function with 1 method)

In [48]:
# benchmark / landscape function


#obj_func(x) = sphere(x)
#obj_func(x) = schwefel_2_22(x)

#obj_func(x) = ackley(x)
#obj_func(x) = rastrigin(x)

obj_func(x) = matyas2(x)
#obj_func(x) = zirilli2(x)

obj_func (generic function with 1 method)

In [14]:
# intensity function

function intensity(N, x, x_prey)
    d = zeros(N)
    S = zeros(N)
    I = zeros(N)
    for i in 1:N-1
        d[i] = euclidean(x[i], x_prey) ^ 2
        S[i] = euclidean(x[i],x[i+1]) ^ 2
    end
    
    d[N] = euclidean(x[N], x_prey)
    S[N] = euclidean(x[N],x[1])
    
    
    
    for i in 1:N
        r2 = rand(Uniform(0,1))
        I[i] = r2 * S[i] / (4 * π * d[i] )
    end
    return I
end

intensity (generic function with 1 method)

In [33]:
# Honey Badger Algorithm

function honey_badger_algorithm(t_max, N, beta, C, x, obj_func)
    # t_max = maximum no of iterations
    # N = population size
    # beta = default value as 6
    # C = default value as 2
    # x = initial population
    # obj_func = benchmark function
    # f = fitness of the population
    # x_prey = best position found so far
    # f_prey = best fitness value so far for the x_prey
    # x_new = new position calculated for the ith honey badger
    # f_new = new fitness calculated for the ith honey badger
    # I = intensity of the honey badger
    # F = disturbance direction
    
    #calucalte the fitness of each honey badger
    f = obj_func.(x)
    #println("fitness vecor: ", f)
    
    #save best posiiton and fitness
    f_prey, x_prey_index = findmin(f)
    x_prey = x[x_prey_index]
    #println("best solution: ", x_prey, " fitness: ", f_prey)
    println("fitness: ", f_prey)
    
    for t in 1:t_max
        
        println("iteration: ", t)
        
        #calculate the density factor / decreasing factor
        alpha = C * exp(-t / t_max)
        #println("alpha: ", alpha)
        
        #calcualte the intensity of the all honey badgers
        I = intensity(N, x, x_prey)
        #println("Intensity: ", I)
        
        for i in 1:N
            
            #println("Honey badger #: ", i)
            
            #calcuate distance between prey and ith honey badger
            d = x_prey - x[i]
            
            #generate disturbance
            r6 = rand(Uniform(0,1))
            if r6<=0.5
                F = 1
            else
                F = -1
            end
            
            
            #generate a random value to deicde between digging vs honey mode
            r2 = rand(Uniform(0,1))
            
            #TODO : if intensity is infinite then go for honey mode
            if isinf(I[i])
                r2 = 0.9
            end
            
            if r2<0.5
                # digging mode
                #println("Digging mode")
                r3 = rand(Uniform(0,1))
                r4 = rand(Uniform(0,1))
                r5 = rand(Uniform(0,1))
                #println("Intensity: ", I[i])
                #p = abs(cos(2*π*r4) * (1 - cos(2*π*r5)))
                #println("cos value: ", p)
                x_new = x_prey + (F * beta * I[i] * x_prey) + (F * r3 * alpha * d * abs(cos(2*π*r4) * (1 - cos(2*π*r5))))
                f_new = obj_func(x_new)
            
            else
                # honey node
                #println("Honey mode")
                r7 = rand(Uniform(0,1))
                x_new = x_prey + (F * r7 * alpha * d)
                f_new = obj_func(x_new)
            end
            
            #TODO: mange xnew to be within the bounds
            
            # evaluate the new posiiton and fitness values
            
            #local position update
            if f_new < f[i]
                #println("local position is improved")
                #println("values", x[i], ";", x_new, ";", f[i], ";", f_new  )
                x[i] = x_new
                f[i] = f_new
                
                #println("local position is improved fin")
            end
            
            #prey position update
            if f_new<f_prey
                println("prey position is improved with honey badger # ", i)
                x_prey = x_new
                f_prey = f_new
            end
            
        end
        println("fitness: ", f_prey)
    end
   return x_prey, f_prey 
end
    

honey_badger_algorithm (generic function with 1 method)

In [49]:
#import Random: seed!

#seed!(0) # set random seed for reproducible results

#dimensions
d = 2
#lower bound
a = -10
#upper bound
b = 10
# population size
N = 50
#iterations
t_max = 400
#constant beta
beta = 6
#constant C
C = 2
#generate honey badgers
x = rand_population_uniform(N, d, a, b)

#println("intitial population: ", x)
x_prey, f_prey = honey_badger_algorithm(t_max, N, beta, C, x, obj_func)
println("best solution: ", x_prey, " fitness: ", f_prey)


fitness: 0.16000000000000014
iteration: 1
prey position is improved with honey badger # 1
fitness: 7.17166188857882e-5
iteration: 2
prey position is improved with honey badger # 46
fitness: 1.616514707097563e-5
iteration: 3
prey position is improved with honey badger # 14
fitness: 1.2845361090375513e-6
iteration: 4
prey position is improved with honey badger # 19
prey position is improved with honey badger # 46
fitness: 9.633055998543956e-7
iteration: 5
prey position is improved with honey badger # 8
prey position is improved with honey badger # 14
prey position is improved with honey badger # 18
prey position is improved with honey badger # 45
fitness: 8.150593803687365e-9
iteration: 6
prey position is improved with honey badger # 30
fitness: 2.9740778757467516e-9
iteration: 7
prey position is improved with honey badger # 14
fitness: 4.916674895985432e-10
iteration: 8
prey position is improved with honey badger # 25
fitness: 4.5922461061748914e-10
iteration: 9
prey position is improve

iteration: 79
prey position is improved with honey badger # 3
prey position is improved with honey badger # 38
fitness: 5.729178189299969e-62
iteration: 80
prey position is improved with honey badger # 32
prey position is improved with honey badger # 38
fitness: 7.789751087462823e-63
iteration: 81
prey position is improved with honey badger # 21
prey position is improved with honey badger # 32
fitness: 5.238478047613826e-63
iteration: 82
prey position is improved with honey badger # 6
prey position is improved with honey badger # 18
prey position is improved with honey badger # 44
prey position is improved with honey badger # 45
fitness: 1.1875605384139014e-64
iteration: 83
prey position is improved with honey badger # 31
prey position is improved with honey badger # 36
prey position is improved with honey badger # 41
fitness: 2.3578540285177225e-66
iteration: 84
prey position is improved with honey badger # 35
fitness: 8.116483725430168e-67
iteration: 85
prey position is improved with

fitness: 1.1492529330176126e-101
iteration: 132
prey position is improved with honey badger # 7
prey position is improved with honey badger # 17
prey position is improved with honey badger # 45
fitness: 3.205530451078762e-103
iteration: 133
prey position is improved with honey badger # 13
prey position is improved with honey badger # 24
fitness: 7.249898049842476e-104
iteration: 134
prey position is improved with honey badger # 2
prey position is improved with honey badger # 17
prey position is improved with honey badger # 25
fitness: 2.061580988478127e-105
iteration: 135
prey position is improved with honey badger # 24
prey position is improved with honey badger # 37
fitness: 6.559678939956758e-106
iteration: 136
prey position is improved with honey badger # 24
prey position is improved with honey badger # 37
fitness: 6.539006585965707e-106
iteration: 137
prey position is improved with honey badger # 12
prey position is improved with honey badger # 33
prey position is improved with ho

prey position is improved with honey badger # 13
prey position is improved with honey badger # 31
fitness: 9.092472087391211e-170
iteration: 201
prey position is improved with honey badger # 12
prey position is improved with honey badger # 31
fitness: 5.277559846433535e-170
iteration: 202
prey position is improved with honey badger # 38
prey position is improved with honey badger # 41
prey position is improved with honey badger # 42
fitness: 9.628735314769889e-171
iteration: 203
prey position is improved with honey badger # 38
prey position is improved with honey badger # 42
fitness: 1.7979947429923327e-171
iteration: 204
fitness: 1.7979947429923327e-171
iteration: 205
prey position is improved with honey badger # 12
prey position is improved with honey badger # 38
fitness: 2.2842249491399065e-172
iteration: 206
prey position is improved with honey badger # 14
prey position is improved with honey badger # 29
prey position is improved with honey badger # 37
fitness: 2.4443324112500026e-

prey position is improved with honey badger # 4
prey position is improved with honey badger # 5
prey position is improved with honey badger # 15
prey position is improved with honey badger # 19
prey position is improved with honey badger # 26
prey position is improved with honey badger # 27
fitness: 2.908632970617673e-218
iteration: 256
prey position is improved with honey badger # 3
prey position is improved with honey badger # 4
prey position is improved with honey badger # 8
prey position is improved with honey badger # 12
prey position is improved with honey badger # 16
prey position is improved with honey badger # 27
prey position is improved with honey badger # 34
fitness: 2.3561644609947283e-220
iteration: 257
prey position is improved with honey badger # 18
prey position is improved with honey badger # 27
prey position is improved with honey badger # 40
fitness: 3.876170602471575e-222
iteration: 258
prey position is improved with honey badger # 6
prey position is improved with 

prey position is improved with honey badger # 17
prey position is improved with honey badger # 26
prey position is improved with honey badger # 31
prey position is improved with honey badger # 37
prey position is improved with honey badger # 41
fitness: 1.6671365012434484e-275
iteration: 308
prey position is improved with honey badger # 12
prey position is improved with honey badger # 37
fitness: 3.513472170066254e-276
iteration: 309
prey position is improved with honey badger # 14
prey position is improved with honey badger # 17
prey position is improved with honey badger # 20
prey position is improved with honey badger # 25
prey position is improved with honey badger # 30
prey position is improved with honey badger # 42
prey position is improved with honey badger # 44
fitness: 8.074469114417176e-277
iteration: 310
prey position is improved with honey badger # 8
prey position is improved with honey badger # 19
prey position is improved with honey badger # 25
prey position is improved 

iteration: 362
fitness: 0.0
iteration: 363
fitness: 0.0
iteration: 364
fitness: 0.0
iteration: 365
fitness: 0.0
iteration: 366
fitness: 0.0
iteration: 367
fitness: 0.0
iteration: 368
fitness: 0.0
iteration: 369
fitness: 0.0
iteration: 370
fitness: 0.0
iteration: 371
fitness: 0.0
iteration: 372
fitness: 0.0
iteration: 373
fitness: 0.0
iteration: 374
fitness: 0.0
iteration: 375
fitness: 0.0
iteration: 376
fitness: 0.0
iteration: 377
fitness: 0.0
iteration: 378
fitness: 0.0
iteration: 379
fitness: 0.0
iteration: 380
fitness: 0.0
iteration: 381
fitness: 0.0
iteration: 382
fitness: 0.0
iteration: 383
fitness: 0.0
iteration: 384
fitness: 0.0
iteration: 385
fitness: 0.0
iteration: 386
fitness: 0.0
iteration: 387
fitness: 0.0
iteration: 388
fitness: 0.0
iteration: 389
fitness: 0.0
iteration: 390
fitness: 0.0
iteration: 391
fitness: 0.0
iteration: 392
fitness: 0.0
iteration: 393
fitness: 0.0
iteration: 394
fitness: 0.0
iteration: 395
fitness: 0.0
iteration: 396
fitness: 0.0
iteration: 397
fitne

In [5]:
x = [1,2,3]
y = [3,1,1]
f =-1
c = 2
a =3

f * c .*x *a

3-element Vector{Int64}:
  -6
 -12
 -18

In [46]:
I = 1/0
if isinf(I)
    println("Bo")
end

Bo


In [63]:
x =[2,3]
matyas2(x)

0.5

In [65]:
+3.38 - 2.88

0.5

In [ ]:
# Honey Badger Algorithm

function honey_badger_algorithm_beta(t_max, N, beta, C, x, obj_func)
    # t_max = maximum no of iterations
    # N = population size
    # beta = default value as 6
    # C = default value as 2
    # x = initial population
    # obj_func = benchmark function
    # f = fitness of the population
    # x_prey = best position found so far
    # f_prey = best fitness value so far for the x_prey
    # x_new = new position calculated for the ith honey badger
    # f_new = new fitness calculated for the ith honey badger
    # I = intensity of the honey badger
    # F = disturbance direction
    
    #calucalte the fitness of each honey badger
    f = obj_func.(x)
    #println("fitness vecor: ", f)
    
    #save best posiiton and fitness
    f_prey, x_prey_index = findmin(f)
    x_prey = x[x_prey_index]
    #println("best solution: ", x_prey, " fitness: ", f_prey)
    println("fitness: ", f_prey)
    
    for t in 1:t_max
        
        println("iteration: ", t)
        
        #calculate the density factor / decreasing factor
        alpha = C * exp(-t / t_max)
        #println("alpha: ", alpha)
        
        #calcualte the intensity of the all honey badgers
        #I = intensity(N, x, x_prey)
        #println("Intensity: ", I)
        
        for i in 1:N
            
            #println("Honey badger #: ", i)
            
            #calcuate distance between prey and ith honey badger
            d = x_prey - x[i]
            
            if i!=N
                di = euclidean(x[i], x_prey) ^ 2
                si = euclidean(x[i],x[i+1]) ^ 2
            else
                di = euclidean(x[i], x_prey) ^ 2
                si = euclidean(x[i],x[1]) ^ 2
            end
            
            r2 = rand(Uniform(0,1))
            I = r2 * si / (4 * π * di )
            
            #generate disturbance
            r6 = rand(Uniform(0,1))
            if r6<=0.5
                F = 1
            else
                F = -1
            end
            
            
            #generate a random value to deicde between digging vs honey mode
            r2 = rand(Uniform(0,1))
            
            #TODO : if intensity is infinite then go for honey mode
            if isinf(I)
                r2 = 0.9
            end
            
            if r2<0.5
                # digging mode
                #println("Digging mode")
                r3 = rand(Uniform(0,1))
                r4 = rand(Uniform(0,1))
                r5 = rand(Uniform(0,1))
                #println("Intensity: ", I[i])
                #p = abs(cos(2*π*r4) * (1 - cos(2*π*r5)))
                #println("cos value: ", p)
                x_new = x_prey + (F * beta * I * x_prey) + (F * r3 * alpha * d * abs(cos(2*π*r4) * (1 - cos(2*π*r5))))
                f_new = obj_func(x_new)
            
            else
                # honey node
                #println("Honey mode")
                r7 = rand(Uniform(0,1))
                x_new = x_prey + (F * r7 * alpha * d)
                f_new = obj_func(x_new)
            end
            
            #TODO: mange xnew to be within the bounds
            
            # evaluate the new posiiton and fitness values
            
            #local position update
            if f_new < f[i]
                #println("local position is improved")
                #println("values", x[i], ";", x_new, ";", f[i], ";", f_new  )
                x[i] = x_new
                f[i] = f_new
                
                #println("local position is improved fin")
            end
            
            #prey position update
            if f_new < f_prey
                println("prey position is improved with honey badger # ", i)
                x_prey = x_new
                f_prey = f_new
            end
            
        end
        println("fitness: ", f_prey)
    end
   return x_prey, f_prey 
end
    